# Model 1: Fully Connected Neural Network

In [ ]:
from pathlib import Path
import numpy as np
import torch 

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
# Import local modules from 'src/utils' as package 'utils'
import sys; sys.path.insert(0, '../')

In [ ]:
import utils

## Read Datasets from .csv

In [ ]:
from utils.file_io import read_angle_datasets

In [ ]:
data_folder = Path("../../data/")
train_data, test_data = read_angle_datasets(data_folder, 0.9)

In [ ]:
input_shape, output_shape = train_data[0][0].shape[0], train_data[0][1].shape[0]
print(f"Data shape {input_shape} / {output_shape} of total {len(train_data)} data rows!")

## Model definition

In [ ]:
from torch import nn, Tensor
from typing import Tuple, List

In [ ]:
class FullyConnected(nn.Module):
    def __init__(self, flattened_input_dim: int, intermediate_dims: List[int], output_dim: int, dropout: float = 0.25, hidden_activation = nn.ReLU) -> None:
        super().__init__()
        self.total_epochs = 0
        self.flatten = nn.Flatten()
        self.hidden = nn.Sequential()
        for i, dim in enumerate(intermediate_dims):
            if i == 0:
                self.hidden.add_module(f"linear_{i+1}", nn.Linear(flattened_input_dim, dim))
            else:
                self.hidden.add_module(f"linear_{i+1}", nn.Linear(intermediate_dims[i-1], dim))

            self.hidden.add_module(f"hidden_activation_{i+1}", hidden_activation())
            self.hidden.add_module(f"dropout_{i+2}", nn.Dropout(dropout))

        self.last = nn.Linear(intermediate_dims[-1], output_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = self.flatten(x)
        x = self.hidden(x)
        return self.last(x)

## Load parameter, functions and dataloader

In [ ]:
import os
from dotenv import load_dotenv
from torch.utils.data import DataLoader

from utils.file_io import save_model
from utils.evaluation import compute_loss_on
from utils.file_io import define_dataloader_from_angle_dataset

In [ ]:
model_path = Path("../../models/fully_connected/").absolute()

In [ ]:
dotenv_path = model_path / ".env"
load_dotenv(dotenv_path=dotenv_path)

learning_rate = float(os.getenv("LEARNING_RATE"))
batch_size = int(os.getenv("BATCH_SIZE"))
num_epochs = int(os.getenv("NUM_EPOCHS"))

In [ ]:
def get_optimizer_function(model: nn.Module, learning_rate: float) -> torch.optim:
    return torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def get_loss_function() -> nn.Module:
    return torch.nn.MSELoss()

In [ ]:
train_dataloader, validation_dataloader, test_dataloader = define_dataloader_from_angle_dataset(train_data, test_data, batch_size, split_size=0.95)

## Define train methods

In [ ]:
def train_epoch(train_dataloader: DataLoader, model: nn.Module, loss_function, optimizer, 
                device: torch.device, report_interval: int = 1000) -> float:

    running_loss = 0
    last_loss = 0
    
    for i, (inputs, true_values) in enumerate(train_dataloader):

        inputs = inputs.to(device)
        true_values = true_values.to(device)
                
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, true_values)
        running_loss += loss
        loss.backward()
        optimizer.step() 
    
    if i % report_interval == report_interval - 1:
        last_loss = running_loss / report_interval
        print(f"batch {i + 1}, Mean Squared Error: {last_loss}")
        running_loss = 0
    
    return last_loss 

In [ ]:
def train(epochs: int, train_dataloader: DataLoader, validation_dataloader: DataLoader, model: nn.Module, loss_function, optimizer, 
          checkpoint_path: Path, device: torch.device = 'cpu', report_interval: int = 1000, tune: bool = False) -> nn.Module:

    best_val_loss = float("inf")

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    model.to(device)

    for epoch in range(model.total_epochs, epochs):
        print(f"Epoch: {epoch + 1}")

        model.train(True)
        avg_loss = train_epoch(train_dataloader, model, loss_function, optimizer, device, report_interval)
        model.eval()

        with torch.no_grad():
            avg_val_loss = compute_loss_on(validation_dataloader, model, loss_function, device=device)

        print(f"Loss on train: {avg_loss}, loss on validation: {avg_val_loss}")

        # Disable checkpoints while running in tune mode
        if avg_val_loss < best_val_loss and not tune:
            best_val_loss = avg_val_loss
            model_path = checkpoint_path / f"{checkpoint_path.name}_{epoch}.pt"
            save_model(model, model_path)

        model.total_epochs += 1
    
    return model   

## Train the model with grid search hyperparameter tuning

In [ ]:
import ray
import json
from ray import tune
from ray import train as ray_train
from ray.train import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from typing import Dict
from utils.evaluation import compute_predictions, compute_losses_from
from functools import partial

In [ ]:
def run_id_generator(data: Dict) -> str:
    return str(hash(json.dumps(data)))

In [ ]:
def parameter_train(parameter: Dict, train_data: DataLoader, test_data: DataLoader, model_input_shape: int, model_output_shape: int, checkpoint_path: Path, device: torch.device) -> None:

    run_id = run_id_generator(parameter)
    run_checkpoint = checkpoint_path / run_id
    run_checkpoint.mkdir(parents=True)

    train_dataloader, validation_dataloader, test_dataloader = define_dataloader_from_angle_dataset(train_data, test_data, batch_size=parameter["batch_size"])

    model = FullyConnected(model_input_shape, parameter["hidden_layers"], model_output_shape)

    optimizer = get_optimizer_function(model, parameter["lr"])
    loss_function = get_loss_function()

    last_model = train(parameter["epochs"], train_dataloader, validation_dataloader, model, loss_function, optimizer, checkpoint_path, device, report_interval=50, tune=True)

    y, y_true = compute_predictions(test_dataloader, last_model)
    test_losses = compute_losses_from(y, y_true, loss_function)

    torch.save((last_model.state_dict(), optimizer.state_dict()), run_checkpoint / "checkpoint.pt")

    print(str(parameter), file=open(run_checkpoint / "trail_parameter.txt", "w"))

    ray_train.report(
        metrics={ "loss": float(test_losses.mean()), "parameter": parameter , "run_id": run_id }
    )

Define parameter ranges

In [ ]:
learning_rate_radius = 1e-3
num_epochs_radius = 5
batch_size_radius = 10
hidden_layer_sizes = [16, 32, 64]
hidden_layer_count = [2, 4, 6]

In [ ]:
parameter = {
    "lr": tune.loguniform(learning_rate_radius - learning_rate, learning_rate + learning_rate_radius),
    "epochs": tune.choice(list(range(num_epochs - num_epochs_radius, num_epochs + num_epochs_radius, 20))),
    "batch_size": tune.choice(list(range(batch_size - batch_size_radius, batch_size + batch_size_radius, 4))),
    "hidden_layers": tune.grid_search(list([hidden_layer_sizes[i % len(hidden_layer_sizes)] for i in range(dim)] for _, dim in enumerate(hidden_layer_count))),
    "dropout": tune.uniform(0.1, 0.5)
}

In [ ]:
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=num_epochs,
    grace_period=1,
    reduction_factor=2,
)

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init(runtime_env={ "py_modules": [utils] })

In [ ]:
ray_resources_manager = tune.with_resources(
    trainable=lambda param: parameter_train(param, train_data, test_data, input_shape, output_shape, model_path, device),
    # See: https://stackoverflow.com/questions/58967793/what-is-the-way-to-make-tune-run-parallel-trials-across-multiple-gpus
    resources={ "cpu": 3, "gpu": 0.25 }
)

tuner = tune.Tuner(
    ray_resources_manager,
    param_space=parameter,
    tune_config=tune.TuneConfig(
        scheduler=scheduler,
        num_samples=10
    )
)

In [ ]:
results = tuner.fit()

In [ ]:
best_result = results.get_best_result("loss", "min")

for key in best_result.metrics["parameter"]:
    print(f"Best Trail: {key} with {best_result.metrics['parameter'][key]}")

In [ ]:
# Checkpoint contains model and optimizer function
best_model, _ = torch.load(model_path / best_result.metrics["run_id"] / "checkpoint.pt")

In [ ]:
print(f"Currently best trial defined by loss: {best_result.metrics['run_id']}", file=open(model_path / "best_trail.txt", "w"))

## Evaluation

In [ ]:
from utils.visualization import create_trace_animation
from matplotlib import pyplot as plt
from IPython.display import HTML

In [ ]:
# Compute eval on the cpu
device = 'cpu'

In [ ]:
%matplotlib notebook
 
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

### Loading the best model

In [ ]:
from utils.file_io import load_model

In [ ]:
loss_function = get_loss_function()

In [ ]:
model = FullyConnected(input_shape, best_result.metrics["parameter"]["hidden_layers"], output_shape)
model.load_state_dict(best_model)
model.eval()

In [ ]:
y, y_true = compute_predictions(test_dataloader, model, device)
test_losses = compute_losses_from(y, y_true, loss_function)
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")

In [ ]:
animation = create_trace_animation(y.numpy(), y_true.numpy())
HTML(animation.to_jshtml())